In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import shapiro, anderson, boxcox
import numpy as np
import math
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Definição de constantes

In [ ]:
l_drop = [#'rotulo',
          'id_reforma',
          'sdv002',
          'sdc002',
          #'mdv001_1',
          #'mdv001_2',
          #'mdv001_3',
          #'mdv001_4',
          'sdu001_1',
          'sdu001_2',
          'TipoAlarmeReleResistencia_id_tipo',
          'TipoAlarmeResistencia_id_tipo',
          'id_reacao','data_hora_reforma'
          ]

d_rename = {'sdp002':'pressure',
            'sdt002':'reactor temperature',
            'sdt003':'pre-reactor temperature',
            'sdu002_1':'humidity',
            'sdu002_2':'local temperature',
            'rotulo': 'label',
            'mdv001_1':'outlet pressure',
            'mdv001_2':'volume flow',
            'mdv001_3':'mass flow',
            'mdv001_4':'outlet temperature'
            }

l_available = ['pressure',
               'reactor temperature',
               'pre-reactor temperature',
               'humidity',
               'local temperature',
               'label']
l_missing = ['outlet pressure',
             'volume flow',
             'mass flow',
             'outlet temperature']

l_new_avaliable = ['number data', 'SMA pressure forecast']

SMA_period  = 10 #


In [ ]:
def recordsMinute(df):

    df['timestamp'] = pd.to_datetime(df['data_hora_reforma'], errors='coerce')
    # Verificando o tipo após a conversão


    df.set_index('timestamp', inplace=True)

    counts_per_minute = df.resample('min').size()

    media_registros_por_minuto = counts_per_minute.mean()

    count = counts_per_minute.describe()
    df = df.reset_index(drop=True)
    return count, df


Leitura dos dados

In [ ]:


#dir = 'BASES/'
dir = '/content/drive/MyDrive/data/BASES/finalVersion/'
dfs = []
days = []
dfs_flow = []
days_flow = []
l_reg_minute = []
for i in (17,21,22,23,24,26,27,30,39):
  df = pd.read_csv(dir+'base'+str(i)+'_final.csv')#'base'+str(i)+'_novo_rotulo.csv')
  day= df['data_hora_reforma'].str[:10][0]
  #print(day)
  days.append(day)
  count, df = recordsMinute(df)#
  l_reg_minute.append(count)#
  df = df.drop(l_drop, axis=1)
  df = df.rename(columns=d_rename)
  df['number data'] = range(len(df))
  df['SMA pressure forecast'] = df['pressure'].shift(-SMA_period).rolling(SMA_period).mean()
  if df['outlet pressure'][0] != -1:
    dfs_flow.append(df)
    days_flow.append(day)
  df = df[l_available + l_new_avaliable]
  dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
df_all_flow = pd.concat(dfs_flow, ignore_index=True)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/BASES/finalVersion/base17_final.csv'

Análise da quantidade de registros por mintuto nas bases.
- A média 2.45, mas a mediana 2.88 e o 3º quartil (75%) teve 3 registros por minuto
- Para fazer uma previsão de 3 minutos, é necessário avançar aproximadamente 8 registros a frente.
- Também é necessário considerar que existem minutos nas bases que não há registros.

In [ ]:
dfCountRegMinute = pd.DataFrame(data=l_reg_minute)
dfCountRegMinute

In [ ]:
df_all.describe()

In [ ]:
for d in dfs:
  print(d.describe())

In [ ]:
df_all_flow.describe()

In [ ]:
def printBoxPlot(df_data, l_remove):
    num_columns = len(df_data.columns) - len(l_remove) # remove label and SMA pressure forecast
    num_rows = math.ceil(num_columns / 3)
    fig, axs = plt.subplots(num_rows, 3, figsize=(15, 5 * num_rows))
    axs = axs.flatten()

    k = 0
    for i, column in enumerate(df_data.columns):
        if column not in l_remove:
            axs[i-k].boxplot(df_data[column].dropna())
            axs[i-k].set_title(f'{column}')
            axs[i-k].set_xlabel(f'{column}')
        else:
            k += 1

    for j in range(i-k+1, len(axs)):
        fig.delaxes(axs[j])

    plt.tight_layout()

    # Exibindo o gráfico
    plt.show()

printBoxPlot(df_data=df_all, l_remove=['label', 'SMA pressure forecast'])


In [ ]:
printBoxPlot(df_data=df_all_flow, l_remove=['pressure', 'reactor temperature', 'pre-reactor temperature', 'humidity', 'local temperature', 'label', 'number data', 'SMA pressure forecast', 'mass flow'])

In [ ]:
def printHistogram(df_data, l_remove):
    num_columns = len(df_data.columns)- len(l_remove)
    num_rows = math.ceil(num_columns / 3)
    fig, axs = plt.subplots(num_rows, 3, figsize=(15, 5 * num_rows))
    axs = axs.flatten()
    k = 0
    for i, column in enumerate(df_data.columns):
        if column not in l_remove:
            axs[i-k].hist(df_data[column].dropna(), bins=30, edgecolor='black')
            axs[i-k].set_title(f'{column}')
            axs[i-k].set_xlabel(f'{column}')
            axs[i-k].set_ylabel('Frequency')
        else:
            k +=1

    # Removendo subplots vazios (se houver)
    for j in range(i-k+1, len(axs)):
        fig.delaxes(axs[j])

    # Ajustando o layout
    plt.tight_layout()

    # Exibindo o gráfico
    plt.show()

printHistogram(df_data=df_all, l_remove=['label', 'SMA pressure forecast'])

In [ ]:
printHistogram(df_data=df_all_flow, l_remove=['pressure', 'reactor temperature', 'pre-reactor temperature', 'humidity', 'local temperature', 'label', 'number data', 'SMA pressure forecast', 'mass flow'])

In [ ]:
skewness = df_all.skew()
kurtosis = df_all.kurtosis()
stats = pd.DataFrame({'Skewness': skewness, 'Kurtosis': kurtosis})
stats


In [ ]:
skewness = df_all_flow.skew()
kurtosis = df_all_flow.kurtosis()
stats = pd.DataFrame({'Skewness': skewness, 'Kurtosis': kurtosis})
stats.head(n=4)


In [ ]:
for column in df_all.columns:
    stat, p = shapiro(df_all[column].dropna())
    print(f'Column: {column}, Stat: {stat:.4f}, p-value: {p}')
    if p > 0.05:
        print('Distribution seems normal (does not reject H0)\n')
    else:
        print('Distribution does not seem normal (rejects H0)\n')

In [ ]:
for column in df_all_flow.columns:
    stat, p = shapiro(df_all_flow[column].dropna())
    print(f'Column: {column}, Stat: {stat:.4f}, p-value: {p}')
    if p > 0.05:
        print('Distribution seems normal (does not reject H0)\n')
    else:
        print('Distribution does not seem normal (rejects H0)\n')

In [ ]:
def plotCorrelation(correlation_matrix):
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True, fmt='.2f')
    plt.title('Matriz de Correlação Geral')
    plt.show()

plotCorrelation(df_all.corr())

In [ ]:
def getAllCorrelation(dfs):
    lCorr = []
    lSamples = []
    for i, df in enumerate(dfs):
        corr = df.corr()
        lCorr.append(corr)
        lSamples.append(df.shape[0])
    return lCorr, lSamples


lCorr, lSample = getAllCorrelation(dfs=dfs)

In [ ]:
def getClusterCorrMatrix(lCorr, lSample):
    flattened_matrices = [matrix.values.flatten() for matrix in lCorr]
    from sklearn.cluster import KMeans

    # Definir o número de clusters desejados
    kmeans = KMeans(n_clusters=4, random_state=42)  # ajuste o número de clusters conforme necessário

    # Treinar o K-means com as matrizes normalizadas
    kmeans.fit(flattened_matrices)

    # Obter os rótulos dos clusters
    labels = kmeans.labels_
    dfCluster = pd.DataFrame({'Base': days, 'Cluster': labels, 'Corr':lCorr, 'Amostras':lSample})
    dfCluster = dfCluster.sort_values(by='Cluster')
    dfCluster = dfCluster.reset_index(drop=True)

    return dfCluster

dfCluster = getClusterCorrMatrix(lCorr, lSample)

In [ ]:
def printCorrelationMatrix(dfCluster):
    plt.figure(figsize=(15, 10))

    # Iterar sobre as bases de dados e plotar os mapas de calor
    j=0

    for i in range(0,dfCluster.shape[0]):
        plt.subplot(3, 3, i + 1)  # Organizando em uma grade 3x3
        #print(df_ordenado['Corr'][i], " ", df_ordenado['Base'][i])
        corr = dfCluster['Corr'][i]

        if i<6:
            if i%3 !=0:
                sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True,
                            xticklabels=[], yticklabels=[],
                            cbar_kws={"shrink": .8}, center=0)
            else:
                sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True,
                            xticklabels=[], yticklabels=df.columns,
                            cbar_kws={"shrink": .8}, center=0)
        else:
            if i%3 !=0:
                sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True,
                            xticklabels=df.columns, yticklabels=[],
                            cbar_kws={"shrink": .8}, center=0)
            else:
                sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True,
                            xticklabels=df.columns, yticklabels=df.columns,
                            cbar_kws={"shrink": .8}, center=0)


        plt.title('Base: ' + dfCluster['Base'][i] + ', samples: '+str(dfCluster['Amostras'][i])+ ", cluster: " + str(dfCluster['Cluster'][i]))
        j+=1

    plt.tight_layout()  # Ajustar o layout para evitar sobreposição
    plt.show()

printCorrelationMatrix(dfCluster)

In [ ]:
plt.figure(figsize=(20, 8))

# Iterar sobre as bases de dados e plotar os mapas de calor
j=0
lCorr = []
lNAmostras = []
for i, df in enumerate(dfs_flow):
    plt.subplot(1, 3, i + 1)  # Organizando em uma grade 3x3
    corr = df.corr()  # Calcular a matriz de correlação
    lCorr.append(corr)
    lNAmostras.append(df.shape[0])
    if i==0:
        sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True,
                xticklabels=df.columns, yticklabels=df.columns,
                cbar_kws={"shrink": .8}, center=0)
    else:
        sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True,
                    xticklabels=df.columns, yticklabels=[],
                    cbar_kws={"shrink": .8}, center=0)


    plt.title('Correlation Matrix - Base: ' + days_flow[i])
    j+=1

plt.tight_layout()  # Ajustar o layout para evitar sobreposição
plt.show()

In [ ]:
dataframes = []

for idx in [17, 21, 22, 23, 24, 26, 27, 30, 39]:
    df = pd.read_csv(f'/content/drive/MyDrive/data/BASES/finalVersion/base{idx}.csv')
    df = df.drop(['number data', 'local temperature', 'SMA pressure forecast'], axis=1)
    dataframes.append(df)

total_df = pd.concat(dataframes, ignore_index=True)

min_values = total_df.min()
max_values = total_df.max()
normalized_dfs = [(df - min_values) / (max_values - min_values) for df in dataframes]

description_map = {0: 'Activation', 1: 'Reaction', 2: 'Irregularity'}

In [ ]:
def plot_data_1(dataset_idx, df, normalized_df):
    plt.figure(figsize=(10, 5))

    colors = plt.cm.get_cmap('tab10', len(normalized_df.columns))

    for idx, col in enumerate(normalized_df.columns):
        if col != 'label':
            plt.plot(normalized_df[col], label=col, color=colors(idx), alpha=0.7)

    previous_value = None
    first_label = True
    for i, value in enumerate(df['label']):
        if first_label:
            plt.text(i + 6, normalized_df.max().max(), description_map[value],
                     rotation=0, verticalalignment='bottom', horizontalalignment='left', fontsize=8, color='black')
            first_label = False
        elif previous_value is not None and value != previous_value:
            plt.axvline(x=i, color='black', linestyle='--', alpha=0.5)
            plt.text(i + 6, normalized_df.max().max(), description_map[value],
                     rotation=0, verticalalignment='bottom', horizontalalignment='left', fontsize=8, color='black')

        previous_value = value

    plt.grid(True, linestyle='--', alpha=0.7)

    plt.title(f'Dataset {dataset_idx:02d}')
    plt.xlabel('Number Data')
    plt.ylabel('Value')
    plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left', fontsize=8)
    plt.tight_layout()
    plt.show()

for idx, (dataset_idx, df, normalized_df) in enumerate(zip([1, 2, 3, 4, 5, 6, 7, 8, 9], dataframes, normalized_dfs)):
  plot_data_1(dataset_idx, df, normalized_df)

In [ ]:
def plot_data_2(dataset_idx, df, normalized_df):
    plt.figure(figsize=(10, 5))

    colors = plt.cm.get_cmap('tab10', len(normalized_df.columns))

    for idx, col in enumerate(normalized_df.columns):
        if col != 'label':
            plt.scatter(range(len(normalized_df[col])), normalized_df[col], label=col, color=colors(idx), s=1, alpha=0.7)

    previous_value = None
    first_label = True
    for i, value in enumerate(df['label']):
        if first_label:
            plt.text(i + 6, normalized_df.max().max(), description_map[value],
                     rotation=0, verticalalignment='bottom', horizontalalignment='left', fontsize=8, color='black')
            first_label = False
        elif previous_value is not None and value != previous_value:
            plt.axvline(x=i, color='black', linestyle='--', alpha=0.5)
            plt.text(i + 6, normalized_df.max().max(), description_map[value],
                     rotation=0, verticalalignment='bottom', horizontalalignment='left', fontsize=8, color='black')

        previous_value = value

    plt.grid(True, linestyle='--', alpha=0.7)

    plt.title(f'Dataset {dataset_idx:02d}')
    plt.xlabel('Number Data')
    plt.ylabel('Value')
    plt.legend(bbox_to_anchor=(1.0, 1), loc='upper left', fontsize=8)
    plt.tight_layout()
    plt.show()

for idx, (dataset_idx, df, normalized_df) in enumerate(zip([1, 2, 3, 4, 5, 6, 7, 8, 9], dataframes, normalized_dfs)):
    plot_data_2(dataset_idx, df, normalized_df)